# 自營商選擇權  
* 方法1: 透過openapi取得資訊  
> [三大法人-選擇權買賣權分計-依日期](https://www.taifex.com.tw/cht/3/callsAndPutsDateView)  
* 方法2: 透過期交所網站下載資料進行更新  
> 首頁 > 交易資訊 > 三大法人 > 下載 > 選擇權買賣權分計 > 依日期  
> [選擇權買賣權分計_依日期下載](https://www.taifex.com.tw/cht/3/callsAndPutsDateDown)


In [1]:
import os
import sys
from pathlib import Path
from datetime import date
from datetime import datetime
import requests
import pandas as pd
import duckdb

In [2]:
from finlab import data
import finlab

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [5]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [6]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 方法1: 透過期交所 open api 取得最近一個交易日數據  
* API只能取得Ｄ-1資訊

In [ ]:
# 取得三大法人-選擇權買賣權分計-依日期
mj_institutional_trader = requests.get(
    "https://openapi.taifex.com.tw/v1/MarketDataOfMajorInstitutionalTradersDetailsOfCallsAndPutsBytheDate", 
    verify=False,
    headers=headers
).json()
print(mj_institutional_trader)

In [ ]:
# json 轉成 dataframe
mj_institutional_trader_df = pd.DataFrame(mj_institutional_trader)
print(mj_institutional_trader_df.shape)
# mj_institutional_trader_df

In [ ]:
# 取得最後一筆交易日，作為方法2的查詢終止日條件
last_txn_date = mj_institutional_trader_df.tail(1)["Date"].values[0]
last_txn_date = pd.to_datetime(last_txn_date)

last_txn_date

### 方法2: 透過期交所網站下載資料進行更新  

In [10]:
# # 讀取台股收盤價資訊
# close = data.get("price:收盤價", save_to_storage=True)

# # 取得近一個交易日期ａｃ
# last_txn_date = close.tail(1).index[0].date()
# print(f"近一個交易日期: {last_txn_date}")

In [ ]:
# 日期參數設定
query_first_date = (last_txn_date - pd.DateOffset(years=3)).strftime("%Y/%m/%d %H:%M")
query_last_date = last_txn_date.strftime("%Y/%m/%d %H:%M")
print(f"{query_first_date} ~ {query_last_date}")

# input_date = "2024/08/05"  # 保留作為手動指定日期
input_date = query_last_date  # 預設指定最近一個交易日期

end_date = pd.to_datetime(input_date)
last_month = (end_date - pd.DateOffset(days=30)).strftime("%Y/%m/%d")

f"{last_month} ~ {input_date}"

In [ ]:
# 設置請求的URL
url = "https://www.taifex.com.tw/cht/3/callsAndPutsDateDown"

# 設置請求的payload
# payload = {
#     "firstDate": "2021/10/04 00:00",
#     "lastDate": "2024/10/04 00:00",
#     "queryStartDate": "2024/10/04",
#     "queryEndDate": "2024/10/04",
#     "commodityId": "TXO"
# }
payload = {
    "firstDate": query_first_date,
    "lastDate": query_last_date,
    "queryStartDate": last_month,
    "queryEndDate": input_date,
    "commodityId": "TXO"
}
print(payload)

# 發送POST請求
response = requests.post(url, headers=headers, data=payload, verify=False)

# 檢查請求是否成功
if response.status_code == 200:
    # 將回應內容保存為CSV文件
    with open("mj_institutional_trader.csv", "wb") as file:
        file.write(response.content)
    print("CSV文件已成功下載並保存為mj_institutional_trader.csv")

    df = pd.read_csv('mj_institutional_trader.csv', encoding='big5')
    df.loc[:,"日期"] = pd.to_datetime(df["日期"])
    # print(df)

    os.remove("mj_institutional_trader.csv")
else:
    print(f"請求失敗，狀態碼：{response.status_code}")

In [ ]:
# 篩選臺指選擇權的自營商資訊
mj_institutional_trader_df = df.copy()

mj_institutional_trader_df = mj_institutional_trader_df[
    (mj_institutional_trader_df["商品名稱"] == "臺指選擇權") & 
    (mj_institutional_trader_df["身份別"] == "自營商")
]
mj_institutional_trader_df = mj_institutional_trader_df[["日期", "買賣權別", "未平倉口數買賣淨額"]]
mj_institutional_trader_df

# 依日期合併未平倉口數買賣淨額, 用買賣權別CALL減掉PUT的未平倉口數買賣淨額值
# 將資料轉換為寬格式，分別顯示CALL和PUT的未平倉口數買賣淨額
mj_institutional_trader_wide = mj_institutional_trader_df.pivot(index="日期", columns="買賣權別", values="未平倉口數買賣淨額").reset_index()

# 重命名列
mj_institutional_trader_wide.columns.name = None
mj_institutional_trader_wide = mj_institutional_trader_wide.rename(columns={"CALL": "CALL未平倉", "PUT": "PUT未平倉"})

# 計算CALL減去PUT的淨值
mj_institutional_trader_wide["淨未平倉"] = mj_institutional_trader_wide["CALL未平倉"] - mj_institutional_trader_wide["PUT未平倉"]

# 只保留日期和淨未平倉列
mj_institutional_trader_df = mj_institutional_trader_wide[["日期", "淨未平倉"]]

# 重命名列為英文，以符合後續使用
mj_institutional_trader_df.columns = ["Date", "net_oi"]

# 顯示結果
print("自營商臺指選擇權未平倉口數買賣淨額（CALL - PUT）：")
print(mj_institutional_trader_df)

## 資料留存ＤＢ

In [14]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [15]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [16]:
table_name = "tw_option_proprietary_traders_oi"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    mj_institutional_trader_df, 
    table_name, 
    date_column='Date'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

In [19]:
# 關閉資料庫連線
conn_duckdb.close()